# Scripting for Rhino, Grasshopper and Dynamo
Both Rhino / Grasshopper and Dynamo have embedded versions of Python. These are special versions of Python that were developed based on the Microsoft dotNET platform and are known as [***IronPython***](http://ironpython.net/). There is [documentation available online](https://ironpython-test.readthedocs.io/en/latest/contents.html).

Microsoft has begun to support Python more actively, and the most recent versions of Visual Studio include Python Tools for Visual Studio (PTVS). There is an older standalone version that can be added into Visual Studio 2015 Express for Desktop.

The version of IronPython that comes with Rhino and Dynamo is v2.7. Details on the differences are available in the [online documentation](https://ironpython-test.readthedocs.io/en/latest/whatsnew/2.7.html#python-3-1-features). However, most Python code is compatible across versions 2.7 and 3.x. There are some libraries available in Python 2.7 that allow you to write your code following Python 3 conventions. The change that affects us most was that the Python 2 print function did not have parentheses and using Python 2 print functions will fail in Python 3:
```python
# Python 2
print "Hello World"

# Python 3
print("Hello World")
```

However, you can load the `__future__` library in Python 2 and load the `print_function`, and you can then continue to write in the Python 3 format. We would recommend importing both the `print_function` and the `division` modules.
```python
# Python 2
from __future__ import print_function, division
print("Hello World")
```
The key difference in terms of results is the handling of division of integers. In Python 2, dividing of integers would generate an integer result. In Python 3, the result is automatically returned as a float when necessary. A double 'divide' (`//`) can be used if an integer division is what is required.
![Comparison of Python 2 & 3](Resources\Images\py23_comparison.png)

## IronPython in Rhino / Grasshopper
It is important to understand that there are a number of scripting languages in Rhino / Grasshopper: 

The original scripting language is RhinoScript, which is based on VBScript. 

1. C#.NET
2. VB.NET
3. Python

McNeel provides a few libraries that can be useful (four of which are listed here). The first two APIs (RhinoCommon and particularly RhinoScriptSyntax) are the main ones that deliver the native functions of Rhino / Grasshopper when you are scripting:
1. [RhinoCommon](https://developer.rhino3d.com/api/RhinoCommon/html/R_Project_RhinoCommon.htm) - a low-level software library for creating and manipulating objects in Rhino (not just geometry, but also the interface and the Rhino documents themselves).
2. [Rhino.Python RhinoScriptSyntax](https://developer.rhino3d.com/api/RhinoScriptSyntax/) - a high-level library that operates on Rhino Objects - mainly geometry. It is best not to do too much mixing of this library with RhinoCommon as the two use different formats (RhinoScriptSyntax uses GUIDs as references, whereas RhinoCommon uses native dotNET objects themselves).
3. [Grasshopper Software Development Kit](https://developer.rhino3d.com/guides/rhinopython/your-first-python-script-in-grasshopper/) - for developing custom components, parameters and data-types for Grasshopper
3. [OpenNURBS](https://www.rhino3d.com/opennurbs) - OpenNURBS libraries allow anyone to read and write the 3DM file format without the need for Rhino ([see information online](https://developer.rhino3d.com/guides/opennurbs/what-is-opennurbs/)).


### IronPython in Rhino
According to McNeel: 
> Python can be used all over Rhino in many different ways. Python can be used to create:
* Interactive scripts.
* New custom commands.
* Create new plug-ins.
* Read and Write customized file formats.
* Interact with cloud applications.
* Create realtime links to other applications
* Create custom Grasshopper components
* Store and display project-specific information beyond what basic Rhino can store.

### Python Rhino / Grasshopper Code Examples
We will follow through the example in (https://developer.rhino3d.com/guides/rhinopython/your-first-python-script-in-grasshopper/)


#### Adding a Point
We will start by adding a range of points using a script in Rhino

1. Start up Rhino
2. Type `EditPythonScript` and press 'enter' or go to `Tools`>`PythonScript`>`Edit...`
3. Cut and paste the following text into the editor and then `Save As...` 'PointArray.py' in the `scripts` folder (default)

```python
import rhinoscriptsyntax as rs
for x in range(10):
    rs.AddPoint((x,0,0))
```
4. Press `Run` (the green triangle).

You can also use Python to creating Rhino Commands - [there is a guide here](https://developer.rhino3d.com/guides/rhinopython/creating-rhino-commands-using-python/).

#### HelloWorld in Grasshopper
We will continue by adding a "HelloWorld" program to Grasshopper, so you will have to 
1. run the `Grasshopper` command in Rhino
2. drag a Python scripting component from the 'Maths' tab to the canvas
3. connect a boolean toggle to the 'x' input (the component is in the 'Params' tab)
4. connect a panel to the 'a' output (the component is in the 'Params' tab)
5. right-click on the component and select 'Open editor...'
6. copy the following code into the editor and press OK
7. Double-click on the toggle to turn to 'True'

```python
if x:
    a = "Hello World!"
else:
    a = "Nothing to say."
```

If you place a second Boolean toggle and add it to the 'x' input (while holding down the 'Shift' key, then you will see both outputs...

*NB If you are using Rhino 5 or earlier and find that you do not have a Grasshopper Script Component, you may have to [install the GHPython plugin from Food4Rhino](https://www.food4rhino.com/app/ghpython).*

![Python in RH & GH](Resources\Images\RH_GH_01.png)


#### Create a line in Grasshopper from two points (x & y)
We can do this two ways:
1. Using **RhinoCommon** - faster, but does not work directly with Grasshopper objects
2. Using **RhinoScriptSyntax** - slower, but acts directly on objects in Grasshopper

##### Using RhinoScriptSyntax
```python
import rhinoscriptsyntax as rs
a = rs.AddLine(x,y)
```

##### Using RhinoCommon with Type Hints
In order to make this work, you have to right-click on the input parameters and define the type hints as Point3d. This is because otherwise the elements are brought in as GUID definitions and Rhino does not know what to do with them.
```python
import Rhino.Geometry as rg
a = rg.Line(x,y)
```

##### Using RhinoCommon without Type Hints
In order to make this work without pre-defining the input (using the 'hint') you will need to 'coerce' Grasshopper into converting the GUID into Rhino geometry.
```python
import Rhino.Geometry as rg
import rhinoscriptsyntax as rs
a = rg.Line(rs.coerce3dpoint(x),rs.coerce3dpoint(y))
```
![Grasshopper Type Hint](Resources\Images\GH_type_hint.png)

### Grasshopper Example - Polygon Generation
The following example is a component that generates a closed polygon with n-sides in the X-Y plane. Inputs are the centre point, the radius of the enclosing circle and the number of sides. 

The code below contains a simple Python function (`nGon()`) that generates a list of triplets that represent 3D coordinates, based on three inputs - the centre point, the radius and the number of sides. This function already exists in Grasshopper (and Dynamo), but we are recreating them as examples.

In [ ]:
from math import pi, sin, cos

def nGon(centre, radius, n_sides):
    x, y, z = centre
    n_s = max(n_sides, 3) # to avoid inputting numbers less than 3
    angles = [2 * pi * i / n_s for i in range(0,int(n_s))]
    list_of_points = [(x + radius * cos(ang), y + radius * sin(ang), z) for ang in angles]
    return list_of_points  # e.g. [[2.4, 5.5, 3,8],[2.7, 5.4, 3,8],[...]]

pts = nGon((1.2, 2.1, 0.0), 4.2, 5)
pts

In [ ]:
closed_pts = pts + [pts[0]]  # have to add the first point to the end to create a closed polyline
closed_pts

We can now look at how we can incorporate this function within the Grasshopper component. We will need to import all the relevant libraries and we will need to convert the 3-tuples (triplets) into Grasshopper points. We will do this using RhinoScriptSyntax.

In terms of incorporating the `nGon()` function, we can either reference it from a file in the same directory, or we can insert the code directly. In the example below we have imported the function from a file called `myGeom.py`.

```python
import rhinoscriptsyntax as rs
from math import sin, cos, pi
from myGeom import nGon  # or add the function internally

pts = [rs.AddPoint(pt) for pt in nGon(rs.coerce3dpoint(C), R, N)]
closed_pts = pts + [pts[0]]  # have to add the first point to the end to create a closed polyline
P = rs.AddPolyline(closed_pts)
```

![Grasshopper Type Hint](Resources\Images\GH_Exercise.png)

The image below shows how the function can be referenced from a file (i.e. myGeom.py). If you have problems getting the function to work, make sure that your indenting is all consistent (tabs OR spaces, but not mixed). Some Python editors will add tabs, and some add 3/4/5 spaces. Python is happy if you are consistent and choose one convention, but not if you mix them.

![Grasshopper Type Hint](Resources\Images\GH_Exercise_02.png)

## Compiling Python Scripts
Note that python scripts can also be compiled into standard Grasshopper components, but only if you are using Rhino 6. THe example below is a custom component that I built to do RC section analysis to the Hong Kong code.

Instructions are available on [the McNeel forum in a posting by Giulio Piacentino](https://discourse.mcneel.com/t/tutorial-creating-a-grasshopper-component-with-the-python-ghpy-compiler/38552).

![Grasshopper Python Compiled Component](Resources\Images\RH_GH_02.PNG)

## Python Scripting in Dynamo
You can do scripting in Dynamo using: 
* [DesignScript](http://designscript.io/DesignScript_user_manual_0.1.pdf) - the native script for Dynamo ([training manual also available](http://primer.dynamobim.org/en/07_Code-Block/7-2_Design-Script-syntax.html)).
* [Python](http://primer.dynamobim.org/en/10_Custom-Nodes/10-4_Python.html)

The Dynamo Primer contains guidance on [scripting Python custom nodes for use with Revit](http://primer.dynamobim.org/en/10_Custom-Nodes/10-5_Python-Revit.html).

Note that it is also possible to drive Revit directly using the API. A Python Revit API wrapper has already been written and is available on GitHub called [RevitPythonShell](https://github.com/architecture-building-systems/revitpythonshell).

### HelloWorld Node in Dynamo

We can create the same HelloWorld node in Dynamo. Dynamo inserts default lines when you first create the Python scripting node.

```python 
import clr
clr.AddReference('RevitAPIUI')
from Autodesk.Revit.UI import TaskDialog

windowTitle = IN[0]
windowMessage = IN[1]

messageDialog = TaskDialog
OUT = messageDialog.Show(windowTitle, windowMessage)
```

![Dynamo Hello World script](Resources\Images\Dynamo_HelloWorld.png)

### Dynamo Example - Polygon Generation

This example is to show how code can be written once and used in multiple locations.

The following is simply a cut and paste job - the same function is being used that was created for Grasshopper - nGon(). Note that we have separated out the logic from the program-specific code. Note that it is different from the Grasshopper-specific code.

```python
import clr
clr.AddReference('ProtoGeometry')
from Autodesk.DesignScript.Geometry import *
#The inputs to this node will be stored as a list in the IN variables.
# dataEnteringNode = IN

# Extract the coordinates of the centre node
x = IN[0].X
y = IN[0].Y
z = IN[0].Z

# Assign the other two nodes to R (radius) & N (number of sides/vertices)
R = IN[1]
N = IN[2]

from math import sin, cos, pi

def nGon(centre, radius, n_sides):
    x, y, z = centre
    n_s = max(n_sides, 3)
    angles = [2 * pi * i / n_s for i in range(0,int(n_s))]
    list_of_points = [(x + radius * cos(ang), y + radius * sin(ang), z) for ang in angles]
    return list_of_points  # e.g. [[2.4, 5.5, 3,8],[2.7, 5.4, 3,8],[...]]

pts = [Point.ByCoordinates(pt[0],pt[1],pt[2]) for pt in nGon([x, y, z], R, N)]

#Assign output to the OUT variable.
OUT = PolyCurve.ByPoints(pts, True)
```
![Dynamo Exercise](Resources\Images\Dynamo_Exercise_01.png)

#### Dynamo Script Referencing External Script
Dynamo Scripts can also reference external scripts. The main requirement is to add the folder containing the script(s) to the path using `sys.path.append()`

```python
import clr
clr.AddReference('ProtoGeometry')
from Autodesk.DesignScript.Geometry import *
import sys
py_path = r"C:\Users\andrew.mole\Arup\Python Training_EA - General\Session 4 - Scripting for Grasshopper and Dynamo"
sys.path.append(py_path)
import math
from myGeom import nGon
# nGon is defined as: nGon(centre, radius, n_sides):
#    returns a list of points  # e.g. [[2.4, 5.5, 3,8],[2.7, 5.4, 3,8],[...]]

#The inputs to this node are stored in the 'IN' variable.
dataEnteringNode = IN

x = IN[0].X
y = IN[0].Y
z = IN[0].Z

R = IN[1]
N = IN[2]

pts = [Point.ByCoordinates(pt[0],pt[1],pt[2]) for pt in nGon([x, y, z], R, N)]

#Output is assigned to the OUT variable.
OUT = PolyCurve.ByPoints(pts, True)
```
![Dynamo Exercise](Resources\Images\Dynamo_Exercise_02.png)

### Exercise
Create a geometrical function and insert it in a Grasshopper 'component' and a Dynamo 'node':
1. A function that creates a grid of lines when provided with a starting point and a list of spacings in x & y direction. You should end up with a grillage defined by a list of lines represented by start and end points (first x-dir, then y-dir and always in the positive direction).
> `gridline(start_point, x_list, y_list)`

For example:
```python
start_point = [1, 2, -1]
x_list = [3,2,2]
y_list = [3,3]

print(gridline(start_point, x_list, y_list))
```
Result is a list of seven pairs of triplets (list of lists of lists)
`[[[1,2,-1],[8,2,-1]], [[1,5,-1],[8,5,-1]], [[1,8,-1],[8,8,-1]], [[1,2,-1],[1,8,-1]], [[4,2,-1],[4,8,-1]], [[6,2,-1],[6,8,-1]], [[8,2,-1],[8,8,-1]]]`